In [ ]:

import os
import random
import numpy as np
import tensorflow as tf

SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:

import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import seaborn as sns
from collections import Counter

In [ ]:
def load_all_splits(base_folder, img_size=(299, 299)):
    print(f"🔍 Loading datasets from: {base_folder}")
    splits = ['train', 'valid', 'test']
    data = {}

    for split in splits:
        print(f"\n📂 Processing '{split}' split")
        folder = os.path.join(base_folder, split)
        X, y, class_names = load_dataset(folder, img_size)
        data[split] = (X, y)

    return data, class_names


In [ ]:
base_folder = r"/content/drive/MyDrive/Gray mold iior/graymold"
data, class_names = load_all_splits(base_folder)


In [ ]:
X_train, y_train = data['train']
X_valid, y_valid = data['valid']
X_test, y_test = data['test']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_class_distribution(labels, class_names, title="Class Distribution"):
    counts = np.bincount(labels)
    plt.figure(figsize=(10, 5))
    sns.barplot(x=class_names, y=counts)
    plt.xticks(rotation=90)
    plt.title(title)
    plt.ylabel("Image Count")
    plt.show()

plot_class_distribution(y_train, class_names, title="Training Set Class Distribution")


In [ ]:
%%time

import os
import time
import subprocess
import numpy as np
from collections import defaultdict
from tensorflow.keras.utils import load_img, img_to_array
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K

# ---------------- Force CPU ---------------- #
tf.config.set_visible_devices([], 'GPU')
print("✅ Running on CPU only (MobileNetV2)")

# ---------------- Load Dataset ---------------- #
# Make sure this is defined before running:
# data, class_names = load_all_splits(base_folder, img_size=(224, 224))
X_train, y_train = data['train']
X_val, y_val = data['valid']

# ---------------- Start Timing ---------------- #
start_time = time.time()

# ---------------- Define Model ---------------- #
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_mobilenet_model.keras', monitor='val_loss', save_best_only=True)

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[-20:]:  # Fine-tune fewer layers for MobileNetV2
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
output_layer = Dense(len(class_names), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.summary()

# ---------------- Train Model ---------------- #
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=8,  # MobileNetV2 is lightweight, so we can afford a larger batch
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# ---------------- End Timing ---------------- #
end_time = time.time()
elapsed_time = end_time - start_time

# ---------------- GPU / CPU Info ---------------- #
def get_device_info():
    try:
        output = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=name,memory.used,memory.total,utilization.gpu,utilization.memory,power.draw', '--format=csv,noheader,nounits']
        ).decode('utf-8').strip()
        stats = output.split(',')
        return {
            'GPU Name': stats[0].strip(),
            'Memory Used (MB)': f"{stats[1].strip()} / {stats[2].strip()}",
            'GPU Load (%)': stats[3].strip(),
            'Memory Load (%)': stats[4].strip(),
            'Power Usage (W)': stats[5].strip()
        }
    except Exception:
        return {"Device": "Running on CPU or GPU not available"}

device_stats = get_device_info()

# ---------------- Final Training Details ---------------- #
final_epoch = len(history.history['accuracy'])
final_acc = history.history['accuracy'][-1]
final_loss = history.history['loss'][-1]

print("\n📊 Device Stats:")
for k, v in device_stats.items():
    print(f"  {k}: {v}")

print("\n🏁 Final Training Details:")
print(f"  Total Epochs: {final_epoch}")
print(f"  Final Loss: {final_loss:.4f}")
print(f"  Final Accuracy: {final_acc:.4f}")

print("\n⏱️ Timing:")
print(f"  CPU Time: {elapsed_time:.2f} seconds")


In [ ]:
%%time
total_params = model.count_params()
trainable_params = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_params = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Non-trainable Parameters: {non_trainable_params:,}")

In [ ]:
import matplotlib.pyplot as plt

def plot_training_curves(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy', marker='o')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy', marker='o')
    plt.legend(loc='lower right')
    plt.title('Accuracy over Epochs')

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss', marker='o')
    plt.plot(epochs_range, val_loss, label='Validation Loss', marker='o')
    plt.legend(loc='upper right')
    plt.title('Loss over Epochs')

    plt.tight_layout()
    plt.show()

# Call after training
plot_training_curves(history)


In [ ]:
%%time
from sklearn.metrics import classification_report, confusion_matrix

# Predict
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = y_test  # <-- Fix here (remove argmax)

# Accuracy
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")
print(f"🧪 Test Loss: {test_loss:.4f}")

# Classification report
print("\n📄 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Predict on test data
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("🧩 Confusion Matrix")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Classification Report
report = classification_report(y_test, y_pred, target_names=class_names, digits=3)
print("📋 Classification Report:\n")
print(report)


In [ ]:
print("🗂️ Class Index Order Used in Training:")
for idx, name in enumerate(class_names):
    print(f"  {idx}: {name}")


In [ ]:
# ---------------- Evaluate on Test Set ---------------- #
X_test, y_test = data['test']

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("\n🧪 Test Set Evaluation:")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Accuracy: {test_accuracy:.4f}")


In [ ]:
model.save('PCM_mobv2.keras')